In [1]:
preparations_already_done = True

In [2]:
if not preparations_already_done:
    # basic
    !pip install numpy scipy scikit-image scikit-learn matplotlib
    # opencv with sift, surf, etc.
    !pip install opencv-contrib-python==3.4.2.16
    !pip install opencv-python==3.4.2.16

In [3]:
if not preparations_already_done:
    # for image augmentation
    !pip install albumentations

In [4]:
if not preparations_already_done:
    # download and unpack
    !mkdir ../data
    !wget -O ../data/database.zip ftp://ftp.uk.research.att.com:pub/data/att_faces.zip
    !unzip -d ../data ../data/database.zip
    !rm ../data/database.zip

In [5]:
import glob
import numpy as np
import matplotlib.pyplot as plt

In [6]:
data_dirs = list(filter(lambda x: 'README' not in x, glob.glob('../data/*')))

In [7]:
from collections import defaultdict as dd

In [8]:
files = dd(list)
for dname in data_dirs:
    files[int(dname.split('/')[-1][1:]) - 1] += glob.glob(dname + '/*')

In [9]:
import cv2

In [10]:
database = np.zeros(
    (
        len(files), 
        len(files[0]),
        *cv2.imread(files[0][0], cv2.IMREAD_GRAYSCALE).shape
    )
)

for k1 in files:
    for k2 in range(len(files[0])):
        database[k1, k2] = cv2.imread(files[k1][k2], cv2.IMREAD_GRAYSCALE)

IndexError: list index out of range

In [11]:
from skimage import exposure, feature
from scipy import fftpack

ModuleNotFoundError: No module named 'skimage'

In [12]:
def get_features(
    img, 
    p1={}, 
    p2={'slice': (slice(20), slice(20)), 'other': {}}, 
    p3={'slice': (slice(20), slice(20)), 'other': {}}, 
    p4={'dsize': (28, 23)},
    p5={'orientations': 8, 'pixels_per_cell': (8, 8), 'cells_per_block': (1, 1)}
):
    dct = fftpack.dct
    f1 = exposure.histogram(img, **p1)[0]
    f2 = np.fft.fft2(img, **p2['other'])[p2['slice']].real.flatten()
    f3 = dct(dct(img.T, **p3['other']).T, **p3['other'])[p3['slice']].real.flatten()
    f4 = cv2.resize(img, **p4).flatten()
    f5 = feature.hog(img, **p5)
    return np.concatenate([f1, f2, f3, f4])

In [13]:
img = database[0, 0]
print(f'{get_features(img).shape[0] / np.prod(img.shape) * 100:.2f}% from original features')

16.50% from original features
